# Benchmark Summary Statistics
---

Description:

    This notebook performs the summary statisticks results for all benchmark functions (one at a time).
    The optimization process is repeated N=50 times (with default settings) and then we collect the results.
    
---

### Import python libraries and set up the project directory

In [8]:
import os, sys
import numpy as np

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

### Import the custom PSO code

In [9]:
from star_pso.population.swarm import Swarm
from star_pso.population.particle import Particle
from star_pso.engines.standard_pso import StandardPSO
from star_pso.utils.auxiliary import cost_function

### Load all the Benchmark problems

In [10]:
# Dimensions: 1D
from star_pso.benchmarks.equal_maxima import EqualMaxima
from star_pso.benchmarks.five_uneven_peak_trap import FiveUnevenPeakTrap
from star_pso.benchmarks.uneven_decreasing_maxima import UnevenDecreasingMaxima

# Dimensions: 2D
from star_pso.benchmarks.himmelblau import Himmelblau
from star_pso.benchmarks.six_hump_camel_back import SixHumpCamelBack

# Dimensions: N >= 2D
from star_pso.benchmarks.shubert import Shubert
from star_pso.benchmarks.vincent import Vincent
from star_pso.benchmarks.rastrigin import Rastrigin

In [11]:
# Select one test problem.
benchmark = Shubert(n_dim=3)

### Define the cost function

Note: All the optimization problems are solved as maximizations.

In [12]:
@cost_function(minimize=False)
def func_benchmark(x_array: np.ndarray, **kwargs) -> float:
    # Return the solution.
    return benchmark.func(x_array).item()
# _end_def_

### Setup the experiment

In [13]:
# Number of repeatitions.
n_repeats = 50

# Swarm population size.
POP_SIZE = 1600

# Maximum function evaluations.
MAX_F_EVALS = 400_000

# Store the results.
results = n_repeats * [0]

# Display Info.
print(f"Testing function: {benchmark.name}\n")

# Go time ...
for i in range(n_repeats):

    # Display current iteration.
    print(f"Running optimization: {i+1} out of {n_repeats} ... ", flush=True)

    # Get an initial (new) sample.
    x_sample = benchmark.sample_random_positions(n_pos=POP_SIZE)

    # Initial swarm population.
    swarm_t0 = Swarm([Particle(x) for x in x_sample])
    
    # Create the StandardPSO object that will carry on the optimization.
    test_PSO = StandardPSO(initial_swarm = swarm_t0, obj_func = func_benchmark,
                           x_min = benchmark.x_min, x_max = benchmark.x_max)
    # Run the optimization.
    test_PSO.run(max_it = 800, f_max_eval = MAX_F_EVALS, verbose = False,
                 options = {"w0": 0.75, "c1": 1.50, "c2": 1.50, "mode": "multimodal"})
    
    # Obtain the number of the global optima that were found for epsilon = 1.0e-4.
    gopt_found, gopt_total = benchmark.search_for_optima(test_PSO.swarm.best_n(POP_SIZE))

    # Update the results.
    results[i] = gopt_found

    # Blank line to declatter the screen.
    print(" ")
# _end_for_

print("Done.")

Testing function: Shubert_2D

Running optimization: 1 out of 50 ... 


11/28/2025 22:01:34 INFO: Initial f_optimal = 108.4742
11/28/2025 22:01:55 WARNING: StandardPSO reached the maximum number of function evaluations at iteration 333


Final f_optimal = 186.7309
run: elapsed time = 21.769 seconds.
 
Running optimization: 2 out of 50 ... 


11/28/2025 22:01:55 INFO: Initial f_optimal = 155.8324
11/28/2025 22:02:11 WARNING: StandardPSO reached the maximum number of function evaluations at iteration 333


Final f_optimal = 186.7309
run: elapsed time = 15.138 seconds.
 
Running optimization: 3 out of 50 ... 


...

Final f_optimal = 186.7309
run: elapsed time = 19.040 seconds.
 
Running optimization: 49 out of 50 ... 


11/28/2025 22:15:01 INFO: Initial f_optimal = 170.2036
11/28/2025 22:15:17 WARNING: StandardPSO reached the maximum number of function evaluations at iteration 333


Final f_optimal = 186.7309
run: elapsed time = 16.863 seconds.
 
Running optimization: 50 out of 50 ... 


11/28/2025 22:15:17 INFO: Initial f_optimal = 161.5035
11/28/2025 22:15:35 WARNING: StandardPSO reached the maximum number of function evaluations at iteration 333


Final f_optimal = 186.7309
run: elapsed time = 18.031 seconds.
 
Done.


In [14]:
# Compute the "Peak Ratio".
peak_ratio = sum(results) / (gopt_total * n_repeats)

print(f"PR = {peak_ratio}")

PR = 0.9833333333333333


In [15]:
# Compute the "Success Rate".
success_rate = sum([r == gopt_total for r in results]) / n_repeats

print(f"SR = {success_rate}")

SR = 0.74


### Optimization Results

Accuracy level: $\epsilon = 1.0^{-4}$

| Benchmark Function | Dimensions | No. Global opt. | MaxFEs  | Population Size | Parameters | PR | SR |
|:---|:---|:---|:---|:---|:---|:---|:---|
| Equal Maxima | 1 | 5 | 50_000| 120 | (0.70, 1.50, 1.50) | 1.0 | 1.0 |
| Five Uneven Peak Trap | 1 | 2 | 50_000 | 120 | (0.70, 1.50, 1.50) | 1.0 | 1.0 |
| Uneven Decreasing Maxima | 1 | 1 | 50_000 | 85 | (0.70, 1.50, 1.50) | 1.0 | 1.0 |
| Himmelblau | 2 | 4 | 50_000 | 100 | (0.70, 1.50, 1.50) |1.0 | 1.0 |
| Six Hump Camel Back | 2 | 2 | 50_000 | 85 | (0.70, 1.50, 1.50) |1.0 | 1.0 |
| Rastrigin | 2 / 4| 6 / 12 | 100_000 / 200_000 | 150 / 400 | (0.70, 1.50, 1.50) |1.000 / 0.998 | 1.000 / 0.980 |
| Shubert | 2 / 3 | 18 / 81 | 200_000 / 400_000 | 600 / 1600 | (0.75, 1.50, 1.50) |0.983 / 0.637 | 0.740 / 0.000 |
| Vincent | 2 / 3 | 36 / 216 | 200_000 / 400_000 | 2000 / 4000 | (0.70, 1.50, 1.50) |0.744 / 0.392 | 0.000 / 0.000 |

### End of file